In [2]:
import pandas as pd 

df = pd.read_csv('csv/food_rows.csv')

In [3]:

df['HOURS']

0               4-6PM
1               2-5PM
2      11:30AM-2:30PM
3              8-10AM
4            4:30-5PM
            ...      
552             3-5PM
553         9:30-11AM
554      8:30-10:30AM
555    2-4PM/10AM-4PM
556          10AM-1PM
Name: HOURS, Length: 557, dtype: object

In [4]:
pd.set_option('display.max_rows', 200)
hours = df['HOURS']
hours.head(200)

0                                 4-6PM
1                                 2-5PM
2                        11:30AM-2:30PM
3                                8-10AM
4                              4:30-5PM
5                           3:30-4:30PM
6                          11AM-12:45PM
7                              11AM-3PM
8                          12:30-3:30PM
9                              9AM-12PM
10                     10AM-1PM & 5-6PM
11                             11AM-1PM
12                             10AM-1PM
13                               12-1PM
14                               8-10AM
15                            10AM-12PM
16                           10-11:30AM
17                               9-11AM
18                            9-10:30AM
19              9-10:30AM & 11:30AM-1PM
20                                5-6PM
21                            9:30-11AM
22                             1:30-3PM
23                         10:30AM-12PM
24                             9AM-12PM


In [5]:
import pandas as pd
import re

# Function to standardize time and convert to 24-hour format
def convert_to_24h(time_str):
    time_str = re.sub(r'\s+', '', time_str)  # Remove spaces
    time_str = time_str.upper()

    # Handle cases like 4-6PM by adding the PM to the end time
    match = re.match(r'(\d{1,2}):?(\d{0,2})?-?(\d{1,2}):?(\d{0,2})?(AM|PM)', time_str)
    if match:
        start_hour = match.group(1)
        start_minute = match.group(2) if match.group(2) else '00'
        end_hour = match.group(3)
        end_minute = match.group(4) if match.group(4) else '00'
        period = match.group(5)

        # Convert start time
        start_hour = int(start_hour)
        if period == 'PM' and start_hour != 12:
            start_hour += 12
        elif period == 'AM' and start_hour == 12:
            start_hour = 0
        start_time = f'{start_hour:02}:{start_minute}'

        # Convert end time, assume same period unless explicitly stated
        end_hour = int(end_hour)
        if period == 'PM' and end_hour != 12:
            end_hour += 12
        elif period == 'AM' and end_hour == 12:
            end_hour = 0
        end_time = f'{end_hour:02}:{end_minute}'

        return f'{start_time} - {end_time}'

    # For cases like 10AM-1PM where the period changes
    time_format = re.findall(r'(\d{1,2}:\d{2}|\d{1,2})(A|P)M', time_str)

    def to_24h(time, period):
        if ':' not in time:
            time += ':00'
        hour, minute = map(int, time.split(':'))
        if period == 'P' and hour != 12:
            hour += 12
        elif period == 'A' and hour == 12:
            hour = 0
        return f'{hour:02}:{minute:02}'

    return ' - '.join([to_24h(time, period) for time, period in time_format])

# Split by '/' and '&', then convert each time range
df['Time_Ranges'] = df['HOURS'].str.split(r'[/&]').apply(lambda times: [convert_to_24h(t) for t in times])

# Expand the list into separate rows
df_expanded = df.explode('Time_Ranges').reset_index(drop=True)

# Further split into Start and End times
df_expanded[['Start_Time', 'End_Time']] = df_expanded['Time_Ranges'].str.split(' - ', expand=True)

# Handle cases where there's no End_Time (single time entries)
df_expanded['End_Time'] = df_expanded['End_Time'].fillna(df_expanded['Start_Time'])

print(df_expanded)

        ID TYPE                                    PROGRAM       ORG PHONE  \
0    80015   FP                 THE BIBLE CHURCH OF CHRIST    718-293-1928   
1    80026   FP         BRONX SEVENTH DAY ADVENTIST CHURCH    646-353-8926   
2    80029   FP  BRONX TEMPLE SEVENTH DAY ADVENTIST CHURCH    718-842-4504   
3    80065   FP                  GETHSEMANE BAPTIST CHURCH    347-948-0772   
4    80081   FP                HOLY TABERNACLE CHURCH INC.  (718) 293-9862   
..     ...  ...                                        ...             ...   
623  87208   FP                  ISAACS HOLMES FOOD PANTRY    212-360-7620   
624  87233   FP                FOREST HILLS SENIORS CENTER    929-349-9740   
625  87341   FP                 CHURCH OF GOD OF SALVATION    718-693-6486   
626  87341   FP                 CHURCH OF GOD OF SALVATION    718-693-6486   
627  87357   FP                     ST. EDWARD FOOD PANTRY  (718) 984-1625   

          DAYS           HOURS  LONGITUDE   LATITUDE  \
0    SU

In [6]:
test = df_expanded[['ID', 'HOURS', 'Time_Ranges', 'Start_Time', 'End_Time']]

In [7]:
test

,ID,HOURS,Time_Ranges,Start_Time,End_Time
0,80015,4-6PM,16:00 - 18:00,16:00,18:00
1,80026,2-5PM,14:00 - 17:00,14:00,17:00
2,80029,11:30AM-2:30PM,11:3 - 00:00,11:3,00:00
3,80065,8-10AM,08:00 - 10:00,08:00,10:00
4,80081,4:30-5PM,16:30 - 17:00,16:30,17:00
...,...,...,...,...,...
623,87208,9:30-11AM,09:30 - 11:00,09:30,11:00
624,87233,8:30-10:30AM,08:30 - 10:30,08:30,10:30
625,87341,2-4PM/10AM-4PM,14:00 - 16:00,14:00,16:00
626,87341,2-4PM/10AM-4PM,01:00 - 00:00,01:00,00:00


In [8]:
df_expanded.columns

Index(['ID', 'TYPE', 'PROGRAM', 'ORG PHONE', 'DAYS', 'HOURS', 'LONGITUDE',
       'LATITUDE', 'FULL ADDRESS', 'BOROUGH', 'ZICODE', 'mon', 'tue', 'wed',
       'thur', 'fri', 'sat', 'sun', 'weeks_open', 'week_1', 'week_2', 'week_3',
       'week_4', 'week_5', 'Time_Ranges', 'Start_Time', 'End_Time'],
      dtype='object')

In [10]:
df.columns

Index(['ID', 'TYPE', 'PROGRAM', 'ORG PHONE', 'DAYS', 'HOURS', 'LONGITUDE',
       'LATITUDE', 'FULL ADDRESS', 'BOROUGH', 'ZICODE', 'mon', 'tue', 'wed',
       'thur', 'fri', 'sat', 'sun', 'weeks_open', 'week_1', 'week_2', 'week_3',
       'week_4', 'week_5', 'Time_Ranges'],
      dtype='object')